# Walk Through for Knowledge Graph Prediction
This notebook outlines the basic process for emulating and predicting a "complete" knowledge graph based on analysis of the observed graph. 
First, we set up the required imports and arguments for the demonstration. This process can be performed all at once from the commandline as well:<br><br>
`python3 predict_kg.py -f FILE -n FNAME -d SNAP_DIR -s -v`<br><br>
(Expectation Maximization iterations and Kronecker graph parameter matrix size are by default 30 and 2 respectively, but can also be set at the commandline with flags `-e` and `-m`

You will need to clone Stanford's [SNAP](https://github.com/snap-stanford/snap) repo to access the `kronem` and `krongen` utilities there. These must also be compiled locally before running. In this example, we have also set an environment variable defining the location of the SNAP folder containing the `kronem` and `krongen` directories.

In [1]:
import networkx as nx
import numpy as np
import os
import pandas as pd
import subprocess
import time

from get_kg_query_params import read_txt
from predict_kg import get_network_params, generate_graph, get_call_str

In [2]:
args_dict = {'file': 'data/train2id.txt',          # Knowledge Graph network file
             'fname': 'new_multivac.txt',          # File to save emulated graph
             'snap_dir': os.environ['SNAP_DIR'],   # Location of SNAP utilities
             'save': 'True',                       # Whether to save the new network to disk
             'verbose': 'True',                    # Whether to print verbose reports on processes
             'em_iter': 2,                         # Expectation Maximization iterations to perform
             'mat_size': 2}                        # Kronecker Graph parameter matrix size (m * m)

Next, we convert our knowledge graph to a simplified version, with relations as directed links connecting head and tail entity nodes.

In [3]:
network = read_txt(args_dict['file'])
network = np.array(network).astype(int)[:,:2]
network = np.unique(network, axis=0)

out_dir, base_name = os.path.split(args_dict['fname'])
base, _ = os.path.splitext(base_name)

new_out = os.path.join(out_dir, "simplified_"+base_name)
np.savetxt(new_out, network, fmt='%u', delimiter='\t')


We then execute the Kronecker Expectation Maximization (KronEM) algorithm on this simplified graph. KronEM was developed at Stanford in 2011 as an efficient method for fitting Kronecker models to large real-world graphs with missing nodes and edges. KronEM uses Expectation Maximization (EM) to fit a model of graph structure using the observed portion, and then estimating the missing portion using the model.

Running in an interpreter or from the command line, we would use the following function to call the `kronem` utility and estimate the Kronecker graph parameters:
```python
params = get_network_params(new_out, 
                            snap_dir=args_dict['snap_dir'], 
                            em_iter=args_dict['em_iter'], 
                            mat_size=args_dict['mat_size'], 
                            verbose=args_dict['verbose'])
```
However, in order to show the process output in the Jupyter environment, we call the utiltiy directly from the notebook here. 

In [4]:
call_str = get_call_str(new_out, args_dict['snap_dir'], em_iter=args_dict['em_iter'], mat_size=args_dict['mat_size'])
!{call_str}

base, _ = os.path.splitext("simplified_"+base_name)

with open("KronEM-{}.tab".format(base), "r") as f:
    params = f.readlines()[-1]

params = params[params.index("[")+1:params.index("]")].replace(",","")
params = '"'+params+'"'


Kronecker graphs. build: 13:43:07, Dec  6 2019. Time: 14:52:42 [Dec  6 2019]
Input graph file (single directed edge per line) (-i:)=/Users/ben_ryan/Documents/git/multivac/simplified_new_multivac.txt
Output file prefix (-o:)=
Initiator matrix size (-n0:)=2
Init Gradient Descent Matrix (R=random) (-m:)=R
Gradient descent iterations for M-step (-gi:)=5
EM iterations (-ei:)=2
Learning rate (-l:)=1e-05
Minimum gradient step for M-step (-mns:)=0.001
Maximum gradient step for M-step (-mxs:)=0.008
Samples for MCMC warm-up (-w:)=8000
Samples per gradient estimation (-s:)=2000
Scale the initiator to match the number of edges (-sim:)=No
Probability of using NodeSwap (vs. EdgeSwap) MCMC proposal (-nsp:)=0.6
Debug mode (-debug:)=No
INIT PARAM
      0.7698      0.5126
      0.4588      0.2017
 (sum:1.9428)
SCALED PARAM
      0.7698      0.5126
      0.4588      0.2017
 (sum:1.9428)

----------------------------------------------------------------------
Fitting graph on 24721 nodes, 400147 edges
Kron


run time: 03m36s (14:56:20)


Having calculated the parameters, we now call `krongen` to generate an emulated graph stochastically. 

In [ ]:
generate_graph(network, params, args_dict['snap_dir'], 
               args_dict['fname'], args_dict['verbose'])


Finally, we merge the observed and emulated graphs to produce a composite predicted graph.

In [ ]:
new_net = read_txt(args_dict['fname'])
new_net = [x for x in new_net if not x[0].startswith("#")]
new_net = np.array(new_net).astype(int)
new_net = np.unique(np.vstack((network, new_net)), axis=0)
np.savetxt(args_dict['fname'], new_net, fmt='%u', delimiter='\t')

In [ ]:
print("Observed network: {} nodes and {} edges.".format(len(np.unique(network)), network.shape[0]))
print("Emulated network: {} nodes and {} edges.".format(len(np.unique(new_net)), new_net.shape[0]))